In [1]:
import sqlite3
import pandas as pd

# Relational Database Management System (RDBMS)

We chose to use an SQLite DBMS to store and manage our extracted data as it allows us to have both CSV and JSON tables that are well structured, organized, and cleaned ahead of time. The code blocks below show how we import and save all of our files into our `.db`file, and then query the database to read-in a specific table into a pandas dataframe.

## Creating the Database

In [2]:
# created out new SQLite database
def drop_unamed_column(df):
    if "Unnamed: 0" in set(df.columns):
        return df.drop("Unnamed: 0", axis=1)
    return df

conn = sqlite3.connect('../Premier_League_Data.db')

# reading-in the acquired data from the extracted CSV files 
player_stats = drop_unamed_column(pd.read_csv('../data/premier_league_player_stats.csv'))
future_matches = drop_unamed_column(pd.read_csv('../data/premier_league_future_matches_05_01_2024.csv')) 
past_matches = drop_unamed_column(pd.read_csv('../data/premier_league_past_matches_05_01_2024.csv'))
articles = drop_unamed_column(pd.read_csv('../data/articles_premier_leagues.csv'))
bets_Betclic = drop_unamed_column(pd.read_csv('../data/betclic_bets_PL_matches.csv'))
bets_Winamax = drop_unamed_column(pd.read_csv('../data/winamax_bets_PL_matches.csv'))

#define the primary keys 
player_stats.reset_index(inplace=True)
future_matches.set_index(['team_home', 'team_away'], inplace=True)
past_matches.set_index(['team_home', 'team_away'], inplace=True)

# writing the data to a table in the SQLite database
future_matches.to_sql('future_matches', conn, if_exists='replace')
past_matches.to_sql('past_matches', conn, if_exists='replace')
articles.to_sql('articles', conn, if_exists='replace')
player_stats.to_sql('player_stats', conn, if_exists='replace')
bets_Betclic.to_sql('bets_Betclic', conn, if_exists='replace')
bets_Winamax.to_sql('bets_Winamax', conn, if_exists='replace')


cursor = conn.cursor()
# Enable foreign keys
cursor.execute('PRAGMA foreign_keys=on;')

"""# Create foreign key constraints
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bets_Betclic (
        team_home TEXT,
        team_away TEXT,
        bet_type TEXT,
        outcome TEXT,
        odd REAL,
        index INTEGER, 
        FOREIGN KEY (team_home, team_away) REFERENCES future_matches
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS bets_Winamax (
        team_home TEXT,
        team_away TEXT,
        bet_type TEXT,
        outcome TEXT,
        odd REAL,
        percentage REAL,
        index INTEGER, 
        FOREIGN KEY (team_home, team_away) REFERENCES future_matches
    );
''')

cursor.execute('''
INSERT INTO bets_Betclic SELECT * FROM bets_Betclic_temp);
''')
cursor.execute('''
INSERT INTO bets_Winamax SELECT * FROM bets_Winamax_temp);
''')

# Disable foreign keys
cursor.execute('PRAGMA foreign_keys=off;')
conn.commit()
"""


conn.close()

## Querying the Database

In [ ]:
conn = sqlite3.connect('../Premier_League_Data.db')


# query to list all tables in the database
all_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(all_tables)

# read a specific table into a pandas DataFrame
df_player_stats = pd.read_sql_query("SELECT * FROM player_stats", conn)
df_betclic = pd.read_sql_query("SELECT * FROM bets_Betclic", conn)

print(df_player_stats)
print(df_betclic)

conn.close()

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('../Premier_League_Data.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON')

# Retrieve the names of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Extract table names from the result
table_names = [table[0] for table in tables]

conn.close()